# Environment

### Env with **Pytorch 1.6** 

# Dataset

In [1]:
import os
os.getcwd()

'/datasets/test/dev/try_module_vision/detectron2_action_recognition'

In [2]:
BASE_PTH = "../"*(len(os.getcwd().split("/"))-1)  #"../../../../../"
DATASET_PTH = BASE_PTH+"datasets/public/HMDB51"

In [3]:
import glob
filenames = sorted(glob.glob(DATASET_PTH+"/*/*.avi"))
len(filenames), filenames[:2]

(6766,
 ['../../../../../datasets/public/HMDB51/brush_hair/April_09_brush_hair_u_nm_np1_ba_goo_0.avi',
  '../../../../../datasets/public/HMDB51/brush_hair/April_09_brush_hair_u_nm_np1_ba_goo_1.avi'])

In [4]:
from video_human_kp_detect import *
import numpy as np

** fvcore version of PathManager will be deprecated soon. **
** Please migrate to the version in iopath repo. **
https://github.com/facebookresearch/iopath 

** fvcore version of PathManager will be deprecated soon. **
** Please migrate to the version in iopath repo. **
https://github.com/facebookresearch/iopath 

/opt/conda/lib/python3.7/site-packages/sklearn/utils/linear_assignment_.py:21: DeprecationWarning: The linear_assignment_ module is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  DeprecationWarning)


# Video2KP

In [5]:
from utils import display_video

- 1100 : [easy] 一人運球
- 600: [easy] 一人攀岩
- 300: [meduim] 一人踢球　一人接球
- 800: [meduim] 一人高空跳下　偵測不佳
- 190 : [hard] 後空翻　偵測不佳

In [6]:
i = 300

In [7]:
display_video(filenames[i])

Moviepy - Building video __temp__.mp4.
Moviepy - Writing video __temp__.mp4

Moviepy - Done !
Moviepy - video ready __temp__.mp4


In [8]:
predictor, cfg = prepare_predictor(THRESH=0.9)
visualizer = prepare_video_visualizer(cfg)

In [9]:
%%time
predict_single_video(predictor, visualizer, 
                     VIDEO_FILE_PTH=filenames[i], 
                     INFO_OUTPUT_PTH="demo/info_dict.joblib", 
                     save_video=True, VIDEO_OUTPUT_PTH="demo/out.avi", 
                     top_k_selected_roles=1,
                     save_video_selected_roles=True, VIDEO_OUTPUT_PTH_SELECTED_ROLES="demo/out_s.avi",
                     tracking_max_distance_limit=[27,50],
                     print_status = True,
                    )

收集 video 內角色的關鍵點..
視覺化關鍵點 並輸出video..


 97%|█████████▋| 33/34 [00:01<00:00, 23.25it/s]

save video [demo/out.avi]
選出 移動最多/最頻繁出現/最完整 的角色 k 個..
  角色[0]出現在 frame [0]
  角色[1]出現在 frame [0]
所有角色 frequency_score : [0.97058824 0.88235294]
所有角色 completeness_score : [0.97058824 0.        ]
所有角色 motion_score : [1.24997499 0.        ]
selected_roles(由分數大到小):  [0]
  selected角色 frequency_score : [0.97058824]
  selected角色 completeness_score : [0.97058824]
  selected角色 motion_score : [1.24997499]
重新篩選出這 k 個角色的關鍵點資訊..
視覺化k 個角色的關鍵點 並輸出video..



 97%|█████████▋| 33/34 [00:01<00:00, 27.65it/s]


save video [demo/out_s.avi]
save video_dict [demo/info_dict.joblib]
CPU times: user 4.52 s, sys: 219 ms, total: 4.74 s
Wall time: 4.77 s


In [10]:
display_video("demo/out.avi")

Moviepy - Building video __temp__.mp4.
Moviepy - Writing video __temp__.mp4

Moviepy - Done !
Moviepy - video ready __temp__.mp4


In [11]:
display_video("demo/out_s.avi")

Moviepy - Building video __temp__.mp4.
Moviepy - Writing video __temp__.mp4

Moviepy - Done !
Moviepy - video ready __temp__.mp4


In [12]:
import joblib
info_d = joblib.load("demo/info_dict.joblib")

In [13]:
info_d.keys()

dict_keys(['width', 'height', 'frames_per_second', 'num_frames', 'num_role_register', 'boxes_buffer', 'kps_buffer', 'roles_buffer', 'frames_split_pos', 'cost_matrix_collection', 'image_size', 'bbox_area_by_role', 'num_frames_by_role', 'motion_distance_by_role', 'frames_by_role', 'completeness_by_role', 'selected_roles', 'boxes_buffer_selected', 'kps_buffer_selected'])

In [14]:
n_frames = info_d['num_frames_by_role']
print(f"影片中有 [{len(n_frames)}] 個人,分別出現的frame數為 {n_frames}")
print(f"選擇的角色id為 { info_d['selected_roles']}")

影片中有 [2] 個人,分別出現的frame數為 [33. 30.]
選擇的角色id為 [0]


# Video2KP - ALL

In [15]:
predictor, cfg = prepare_predictor(THRESH=0.9)
visualizer = prepare_video_visualizer(cfg)

In [16]:
SAVE_KPS_PTH = "predicted_kps" 

In [17]:
import subprocess 
if not os.path.isdir(SAVE_KPS_PTH):
    subprocess.run("mkdir "+SAVE_KPS_PTH, shell=True)#若沒有此路徑 則新增
else:
    subprocess.run("rm "+SAVE_KPS_PTH+"/*", shell=True)#若有此路徑 清空路徑下所有檔案

In [18]:
import joblib, time

In [19]:
st = time.time()
for j, fn in enumerate(filenames):

    st_ = time.time()
    class_ = fn.split("/")[-2] #video_info_df.loc[j,"class"]
    id_ = str(j).zfill(4)
    INFO_OUTPUT_PTH=f"{SAVE_KPS_PTH}/[{id_}]{class_}.joblib"
    txt = f"[{(j+1)}/{len(filenames)}] {INFO_OUTPUT_PTH}"
    if not os.path.isfile(INFO_OUTPUT_PTH):
        try:
            predict_single_video(predictor, visualizer, 
                                 VIDEO_FILE_PTH=fn, 
                                 INFO_OUTPUT_PTH=INFO_OUTPUT_PTH, 
                                 save_video=False, #VIDEO_OUTPUT_PTH="out.avi", 
                                 top_k_selected_roles=1,
                                 save_video_selected_roles=False, #VIDEO_OUTPUT_PTH_SELECTED_ROLES="out_s.avi",
                                 tracking_max_distance_limit=[27,50],
                                 print_status = False,
                                ) 
            print("{}  Spent Time [{:.2f}] sec".format(txt, time.time()-st_)) 
        except:
            print(f"{txt}  Error!")
    else:
        print(f"{txt}  Already Existed!")
print("Total Spent Time [{:.2f}] sec".format(time.time()-st)) 

[1/6766] predicted_kps/[0000]brush_hair.joblib  Spent Time [24.20] sec
[2/6766] predicted_kps/[0001]brush_hair.joblib  Spent Time [23.31] sec
[3/6766] predicted_kps/[0002]brush_hair.joblib  Spent Time [19.09] sec
[4/6766] predicted_kps/[0003]brush_hair.joblib  Spent Time [17.08] sec
[5/6766] predicted_kps/[0004]brush_hair.joblib  Spent Time [11.02] sec
[6/6766] predicted_kps/[0005]brush_hair.joblib  Spent Time [9.63] sec
[7/6766] predicted_kps/[0006]brush_hair.joblib  Spent Time [25.11] sec
[8/6766] predicted_kps/[0007]brush_hair.joblib  Spent Time [13.39] sec
[9/6766] predicted_kps/[0008]brush_hair.joblib  Spent Time [25.66] sec
[10/6766] predicted_kps/[0009]brush_hair.joblib  Error!
[11/6766] predicted_kps/[0010]brush_hair.joblib  Spent Time [38.23] sec
[12/6766] predicted_kps/[0011]brush_hair.joblib  Spent Time [15.39] sec
[13/6766] predicted_kps/[0012]brush_hair.joblib  Spent Time [4.63] sec
[14/6766] predicted_kps/[0013]brush_hair.joblib  Error!
[15/6766] predicted_kps/[0014]brush